<img src="https://unidata.ucar.edu/images/logos/badges/badge_unidata_100.jpg" alt="Unidata Logo" style="float: right; height: 98px;">

# Siphon THREDDS Jupyter Notebook Viewer 

## Dataset: {{datasetName}}
___

### Dependencies:
* *Siphon*:`pip install siphon`
* *matplotlib*:`pip install matplotlib` or `conda install -c conda-forge matplotlib`
* *ipywidgets*:`pip install ipywidgets` or `conda install -c conda-forge ipywidgets`  
* enable *ipywidgets*:
    * using Juputer Notebooks: `jupyter nbextension enable --py widgetsnbextension`
    * using JupyterLab:
        * nodejs: `conda install nodejs`
        * `jupyter labextension install @jupyter-widgets/jupyterlab-manager`
* *numpy*: `pip install numpy` or `conda install numpy`
___

In [ ]:
from siphon.catalog import TDSCatalog
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets

In [ ]:
catUrl = "{{catUrl}}";
datasetName = "{{datasetName}}";

### Access a dataset
With the TDS catalog url, we can use Siphon to get the dataset named `datasetName`.

In [ ]:
catalog = TDSCatalog(catUrl)

In [ ]:
ds = catalog.datasets[datasetName]
ds.name

Datasets each have a set of access protocols:

In [ ]:
list(ds.access_urls)

Siphon's `remote-access` returns a `Dataset` object, which opens the remote dataset and provides access to its metadata:

In [ ]:
dataset = ds.remote_access()

In [ ]:
list(dataset.ncattrs())

### Display a variable:
1) Create a widget to select from a list of all variables in this dataset.


In [ ]:
var_name = widgets.RadioButtons(
    options=list(dataset.variables),
    description='Variable:')


2) Run the cell below to display the widget.  
3) Select the variable you wish to view.

In [ ]:
display(var_name)

4) Display information about the chosen variable

In [ ]:
var = dataset.variables[var_name.value];
print("Name: " + var.name);
print("Dimensions: " + str(list(var.dimensions)));
print("Shape: " + str(var.shape));
from functools import reduce
nelems = reduce(operator.mul, var.shape, 1);
print("# elements: " + str(nelems));
print("Datatype: " + str(var.dtype);

5) Attempt to plot the variable.

In [ ]:
# EDIT these values to print or plot fewer/more elements
max_print_elems = 1000; # don't print more than this number of elements
max_elems = 10000; # don't request more than this number of elements (<100M to avoid HTTP errors)

# only atttempt to plot numeric types
if (not(var.dtype == np.uint8 or np.can_cast(var.dtype, float, "same_kind"))):
    print("Not a numeric type - cannot plot variable: ", var.name);
    if (nelems > max_print_elems):
        print("Too many elements - printing first " + str(max_print_elems) + " elements");
        var = var.flatten()[0:max_print_elems];
        print(var);
        
else:
    # assure plotable number of dimensions
    ndims = len([s for s in var.shape if s > 1]);
    max_dims = 2;
    shape = np.array(var.shape);
    if (ndims > max_dims):
        print("Too many dimensions - reducing last " + str(ndims-max_dims) + " dimensions.")
        shape[np.argwhere(shape>1).flatten().tolist()[max_dims:]] = 1;
        print("New shape: " + str(shape))
        ndims = max_dims;

    # assure plotable number of elements   
    nelems = reduce(operator.mul, shape, 1);
    scale = (nelems/max_elems)**(1/ndims) if ndims else 0;
    if (scale > 1):
        print("Too many elements - subsetting variable")
        shape[np.argwhere(shape>1).flatten().tolist()] = shape[np.argwhere(shape>1).flatten().tolist()]//scale;
        print("New shape: " + str(shape));
              
    # EDIT indices variable to change the number/indices of plotted elements
    to_slice = lambda x : slice(None, x, None)
    indices = [to_slice(s) for s in shape]
    
    # make copy of (possible) subset of variable
    disp_var = var[indices];

    # plot the variable
    %matplotlib inline
    # for one-dimensional data, print value
    if (ndims == 0):
        print(var.name, ": ", disp_var)
    # for two-dimensional data, make a line plot
    elif (ndims == 1):
        plt.plot(np.squeeze(np.array([range(len(np.squeeze(disp_var[:])))])), np.squeeze(disp_var[:]), 'bo', markersize=5)
        plt.title(var.name)
        plt.show()
    # for three-dimensional data, make an image
    elif (ndims == 2):
        plt.imshow(np.squeeze(disp_var[:]))
        plt.title(var.name)
        plt.show()

**Note** that data are only transferred over the network when the variable is sliced, and only data corresponding to the slice are downloaded. In this case, we are ask for a subset of the data with `disp_var = var[indices]`. You may change the values to `indices` to request a different subset of data.

---

6) To plot a different variable, select it in the widget and rerun the subsequent cells.

### More with Siphon
To see what else you can do, view the [Siphon API](https://unidata.github.io/siphon/latest/api/index.html).

In [ ]:
### Your code here ###